# 3D X-ray Histology VISualisation (3DXRH-Vis)

3DXRH-Vis is a tool for interactive visualisation of 2D images registered to a 3D dataset. It was designed for classical histology slides registered to 3D X-ray micro-computed tomography datasets, but should also work for any 2D-3D registered images.


In [4]:
# Set up, only needs to be run once at beginning
!conda activate 3DXRH_Vis

import os
import numpy as np
from PIL import Image
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
from ipyfilechooser import FileChooser
from skimage import io, viewer, util

## Explore 3D dataset

3D images are displayed as sequences of 2D images. Run the following cell to import a 3D dataset.

In [5]:
# Import a 3D dataset
fc_3d = FileChooser(os.getcwd())
display(fc_3d)

FileChooser(path='C:\Users\emlh1n13\OneDrive - University of Southampton\Data\2019-20\3DXRH_Vis', filename='',…

Run the following cell to view the 3D dataset.

In [ ]:
def show_3D_slider():
    img = Image.open(fc_3d.selected)
    
    def show_slice(slice_number):
        img.seek(slice_number)
        display(img)
        
    slider = widgets.IntSlider(min=0, max=img.n_frames, step=1, description="Slice number")
    interact(show_slice, slice_number=slider)
    
    return img

img = show_3D_slider()

## slower version of slider view for 3D image, but has entire image in memory.

In [6]:
img_import = io.imread(fc_3d.selected, plugin='tifffile')
img = util.img_as_ubyte(img_import) # 8bit RGB

In [27]:
def show_3D_slider(img):
    def show_slice(slice_number):
        io.imshow(img[slice_number])
        io.show()
    slider = widgets.IntSlider(min=0, max=len(img), step=1, description="Slice number")
    interact(show_slice, slice_number=slider)

show_3D_slider(img)

interactive(children=(IntSlider(value=0, description='Slice number', max=239), Output()), _dom_classes=('widge…

## Import registered 2D images into 3D stack.

Run the following cell to import a 2D image and specify the position in the 3D stack to import the 2D image.

In [16]:
# Import a 2D dataset
fc_2d = FileChooser(os.getcwd())
display(fc_2d)

# position_2d = widgets.BoundedFloatText(min=0, max=img.n_frames, step=1, description="Position of 2D image") if loaded with PIL
position_2d = widgets.BoundedFloatText(min=0, max=len(img), step=1, description="Position of 2D image")
display(position_2d)

FileChooser(path='C:\Users\emlh1n13\OneDrive - University of Southampton\Data\2019-20\3DXRH_Vis', filename='',…

BoundedFloatText(value=0.0, description='Position of 2D image', max=239.0, step=1.0)

The following cell inserts the 2D image at the specified position.

In [31]:
# Import 2D image and insert into stack
img_2d_import = io.imread(fc_2d.selected, plugin='tifffile')
img_2d = util.img_as_ubyte(img_2d_import) # 8bit RGB
img_combined = np.insert(img, [int(position_2d.value)], img_2d, axis=0)
print("2D image {} imported into 3D image {} at position {}".format(fc_3d.selected_filename, fc_2d.selected_filename, str(position_2d.value)))

# Show combined stack
show_3D_slider(img_combined)

2D image HN1_test.tif imported into 3D image HN1_201_test.tif at position 201.0


interactive(children=(IntSlider(value=0, description='Slice number', max=240), Output()), _dom_classes=('widge…

# to do

- Wrap 2D importing into a function so we can import several 2D images
- Menu to choose 2D images to skip to
- Side by side viewer
- Linked pan and zoom for side-by-side
- Max intensity
- Orthoslice viewer for combined stack
- Port to Voila